In [1]:
import warnings
warnings.filterwarnings("ignore")
from torch.optim import SGD
import albumentations as A
import os
import torch

from going_modular.dataloader.multitask import create_multitask_datafetcher
from going_modular.model.MTLFaceRecognition import MTLFaceRecognition
from going_modular.loss.MultiTaskLoss import MultiTaskLoss
from going_modular.train_eval.train import fit
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.utils.PolynomialLRWarmup import PolynomialLRWarmup
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint


device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'albedo',
    
    # Thư mục
    'dataset_dir': './Dataset',
    'checkpoint_dir': './checkpoint/multi/',
    
    # Cấu hình train
    'device': device,
    'epochs': 299,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'base_lr': 0.2,
    
    # Cấu hình network
    'backbone': 'miresnet18',
    'embedding_size': 512,
    'num_classes': None,
    'loss_spectacles_weight': 0.01,
    'loss_da_spectacles_weight': 0.01,
    'loss_occlusion_weight': 0.01,
    'loss_da_occlusion_weight': 0.01,
    'loss_facial_hair_weight': 0.01,
    'loss_da_facial_hair_weight': 0.01,
    'loss_pose_weight': 0.01,
    'loss_da_pose_weight': 0.01,
    'loss_gender_weight': 0.01,
    'loss_da_gender_weight': 0.01,
    'loss_emotion_weight': 0.01,
    'loss_da_emotion_weight': 0.01,
}

CONFIGURATION['num_classes'] = len(os.listdir('./Dataset/Albedo/train'))

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})

train_dataloader, test_dataloader, train_weight_class = create_multitask_datafetcher(CONFIGURATION, train_transform, test_transform)
model = MTLFaceRecognition(CONFIGURATION['backbone'], CONFIGURATION['num_classes'])

criterion = MultiTaskLoss(os.path.join(CONFIGURATION['dataset_dir'], 'train_set.csv'), CONFIGURATION)
optimizer = SGD(model.parameters(), lr=CONFIGURATION['base_lr'])
scheduler = PolynomialLRWarmup(optimizer, warmup_iters=10, total_iters=CONFIGURATION['epochs'], power=2)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      
checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

2025-01-13 19:03:00.696154: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-13 19:03:01.411851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    train_dataloader=train_dataloader, 
    test_dataloader=test_dataloader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
	train: loss: 3.0464 | auc_cos: 0.6881 | auc_eu: 0.7132
	test: auc_cos: 0.6458 | auc_eu: 0.5708
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/3d_face_recognition/checkpoint/multi/albedo/models/checkpoint.pth
Epoch 2:
	train: loss: 2.3217 | auc_cos: 0.7915 | auc_eu: 0.7691
	test: auc_cos: 0.6777 | auc_eu: 0.6440
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/3d_face_recognition/checkpoint/multi/albedo/models/checkpoint.pth


KeyboardInterrupt: 